In [ ]:
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text, inspect

from postgres_auth import db_writer_url


load_dotenv()


engine = create_engine(db_writer_url)

with engine.connect() as conn:
    result = conn.execute(text("SELECT MAX(obs_time) FROM webpage_plot_data"))
    latest_history = result.scalar()

lower_bound = latest_history if latest_history else '1970-01-01'

query = f"""
    SELECT sensor_id, obs_time, temp1, hum1, temp2, hum2, co2, aqi
    FROM observations
    WHERE obs_time > '{lower_bound}'
"""
df = pd.read_sql_query(query, engine)